# Other neural network examples

Here we will show a couple of other examples for neural networks, these are more advanced though

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Conv2D, Flatten, Reshape, Conv2DTranspose, Activation, LSTM, Conv1D, GlobalMaxPooling1D, Embedding
from keras.callbacks import TensorBoard, ModelCheckpoint
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing import sequence

from keras.datasets import imdb

%matplotlib inline

## Sequence prediction with LSTM

Neural networks can also work with sequences. Sequences are typically modeled with Recurent Neural Networks. These have connections between neurons within one layer, allowing them to preserve state from before and use it in the next prediction. Lets load and explore some data first.

You can find more datasets at https://datamarket.com/data/list/?q=cat:g22%20provider:tsdl

In [ ]:
!wget https://raw.githubusercontent.com/hlavacek/openslava-ai-workshop-2018/master/5_other_examples/datasets/normalized_apple_prices.csv
!wget https://raw.githubusercontent.com/hlavacek/openslava-ai-workshop-2018/master/5_other_examples/datasets/ibm-common-stock-closing-prices-.csv

In [ ]:
# dataset = np.loadtxt('normalized_apple_prices.csv')
dataset = np.loadtxt('ibm-common-stock-closing-prices-.csv', usecols=(1,), skiprows=1, delimiter=',')

We need to scale the data into interval 0 - 1, to help the neural network optimize for the data

In [ ]:
dataset = dataset.reshape(-1, 1)
# dataset_2.shape
scaler = MinMaxScaler()
scaler.fit(dataset)
dataset = scaler.transform(dataset)

In [ ]:
# lets take a look at our time series
plt.plot(dataset)
plt.xlabel('time period')
plt.ylabel('normalized series value')

We need to preprocess our dataset, with the following idea in mind:
1. The neural network will take small part of the sequence (e.g. 7 continuous values) as input
2. It will predict the next value based on this window

Therefore we take the dataset, make the windows and expected predictions for each window (the next number in the sequence)

In [ ]:
window_size = 7
X = np.array([dataset[i:i+window_size] for i in range(0, len(dataset) - window_size)])
y = np.array([dataset[i] for i in range(7, len(dataset))])

In [ ]:
print(X[0])
print(y[0])

In [ ]:
print(X[2])
print(y[2])

In [ ]:
train_test_split = int(np.ceil(2*len(y)/float(3)))   # set the split point

# partition the training set
X_train = X[:train_test_split]
y_train = y[:train_test_split]

# keep the last chunk for testing
X_test = X[train_test_split:]
y_test = y[train_test_split:]

# reshape the imput for LSTM layer [samples, window size, stepsize] 
X_train = np.asarray(np.reshape(X_train, (X_train.shape[0], window_size, 1)))
X_test = np.asarray(np.reshape(X_test, (X_test.shape[0], window_size, 1)))

In [ ]:
model = Sequential()

model.add(LSTM(5, input_shape=(window_size, 1)))
model.add(Dense(1))


In [ ]:
# build model using keras documentation recommended optimizer initialization
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# compile the model
model.compile(loss='mean_squared_error', optimizer=optimizer)

In [ ]:
model.fit(X_train, y_train, epochs=1000, batch_size=50, verbose=1)

In [ ]:
# generate predictions for training
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
# print out training and testing errors
training_error = model.evaluate(X_train, y_train, verbose=0)
print('training error = ' + str(training_error))

testing_error = model.evaluate(X_test, y_test, verbose=0)
print('testing error = ' + str(testing_error))

In [ ]:
### Plot everything - the original series as well as predictions on training and testing sets
import matplotlib.pyplot as plt
%matplotlib inline

# plot original series
plt.plot(dataset,color = 'k')

# plot training set prediction
split_pt = train_test_split + window_size 
plt.plot(np.arange(window_size,split_pt,1),train_predict,color = 'b')

# plot testing set prediction
plt.plot(np.arange(split_pt,split_pt + len(test_predict),1),test_predict,color = 'r')

# pretty up graph
plt.xlabel('day')
plt.ylabel('(normalized) price of stock')
plt.legend(['original series','training fit','testing fit'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## Text classification from IMDB dataset

Another task where you can use neural networks is a text classification (or text processing). For this example we are going to use the IMDB dataset, which is also part of Keras. This dataset contains IMDB reviews and tells whether they are positive or negative. Lets load the dataset

In [ ]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

How does the data look like?

In [ ]:
print(x_train[0])
print("label: {}".format(y_train[0]))

You can see we have received just an array of numbers, nothing else. These are actually indexes of words, where the words are indexed by how most common they are in the whole dataset. So index 4 means that this was the 4th most common word in the dataset. The labels are either 0 or 1.

In [ ]:
# Hyperparameters
max_features = 5000
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

Because the sequences have different lengths, we need to pad them to be on the same length (400 words in our case). For this Keras has some helper functions:

In [ ]:
maxlen = 400
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
print(x_train[1])

Now we will define our neural network model:

In [ ]:
model = Sequential()

As a firsts layer we will have an Embedding layer. This is a very interesting one, because its purpose is to learn good mathematical representation of the words. But good for what? Well good for our specific task and good for our neural network - in a different neural network or with different data, the Embedding layer will actually learn something else.

In [ ]:
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))

# Dropout layer to avoid overfitting
model.add(Dropout(0.2))

The next layers should learn to capture the relations between the words. For this we can use a dense layer (has many connections, so slow learning), LSTM (also quite slow to learn), or a convolutional layer. This is similar like in the image classification, but we will have just 1D convolution:

![1D Convolution](conv1d.png)

In [ ]:
# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

Next we'll add a couple of standard dense layers and dropouts. Please note that the last layer is just sigmoid layer - we'll have only 0 or 1 as output

In [ ]:
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

Lets train the network, two epochs are quite enough

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

We can now do some predictions:

In [ ]:
model.predict(x_test[0:5])

In [ ]:
print(y_test[0:5])